## Step 4 - Climate App

Now that you have completed your initial analysis, design a Flask api based on the queries that you have just developed.

* Use FLASK to create your routes.

### Routes

* `/api/v1.0/precipitation`

  * Query for the dates and temperature observations from the last year.

  * Convert the query results to a Dictionary using `date` as the key and `tobs` as the value.

  * Return the json representation of your dictionary.

* `/api/v1.0/stations`

  * Return a json list of stations from the dataset.

* `/api/v1.0/tobs`

  * Return a json list of Temperature Observations (tobs) for the previous year

* `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`

  * Return a json list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.

  * When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.

  * When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` for dates between the start and end date inclusive.

## Hints

* You will need to join the station and measurement tables for some of the analysis queries.

* Use Flask `jsonify` to convert your api data into a valid json response object.

In [1]:
import datetime 
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [2]:
connection_string = 'sqlite:///hawaii.sqlite'
engine = create_engine(connection_string)

In [3]:
Base = automap_base()

In [4]:
Base.prepare(engine, reflect = True)

In [5]:
Hawaii = Base.classes.hawaii_data
session = Session(engine)

In [6]:
first_row = session.query(Hawaii).first()

In [7]:
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1147bdb38>,
 'date_format': datetime.datetime(2010, 1, 1, 0, 0),
 'day': 1,
 'elevation': 3,
 'id': 1,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'month': 1,
 'name': 'WAIKIKI 717.2, HI US',
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65,
 'year': 2010}

In [77]:
#Query to return one year's worth of prcp data.
#Function does a groupby for each day and sums across all four stations.

#https://stackoverflow.com/questions/25558582/groupby-and-sum-in-sqlalchemy
#https://stackoverflow.com/questions/8895208/sqlalchemy-how-to-filter-date-field


prcp_query = session.query(Hawaii.date_format, func.sum(Hawaii.prcp)).\
            filter(Hawaii.date_format.between('2016-08-23 00:00:00.000000', '2017-08-23 00:00:00.000000'))\
            .group_by(Hawaii.date_format).all()
prcp_query

[(datetime.datetime(2016, 8, 23, 0, 0), 2.7578461538461543),
 (datetime.datetime(2016, 8, 24, 0, 0), 9.377846153846152),
 (datetime.datetime(2016, 8, 25, 0, 0), 0.54),
 (datetime.datetime(2016, 8, 26, 0, 0), 0.09999999999999999),
 (datetime.datetime(2016, 8, 27, 0, 0), 0.763960396039604),
 (datetime.datetime(2016, 8, 28, 0, 0), 3.0999999999999996),
 (datetime.datetime(2016, 8, 29, 0, 0), 1.5078461538461538),
 (datetime.datetime(2016, 8, 30, 0, 0), 0.07),
 (datetime.datetime(2016, 8, 31, 0, 0), 3.333193376068376),
 (datetime.datetime(2016, 9, 1, 0, 0), 0.05728571428571429),
 (datetime.datetime(2016, 9, 2, 0, 0), 0.38728571428571434),
 (datetime.datetime(2016, 9, 3, 0, 0), 1.27),
 (datetime.datetime(2016, 9, 4, 0, 0), 1.3800000000000001),
 (datetime.datetime(2016, 9, 5, 0, 0), 0.48662873794491085),
 (datetime.datetime(2016, 9, 6, 0, 0), 1.277192118226601),
 (datetime.datetime(2016, 9, 7, 0, 0), 2.0),
 (datetime.datetime(2016, 9, 8, 0, 0), 0.4872857142857143),
 (datetime.datetime(2016, 9,

SyntaxError: unexpected EOF while parsing (<ipython-input-78-d22d1e42db4c>, line 4)

In [44]:
#Query to return the most active stations.
#http://docs.sqlalchemy.org/en/latest/orm/query.html

active_query = session.query(Hawaii.name, func.count(Hawaii.tobs)\
                            ).group_by(Hawaii.name\
                                      ).order_by(Hawaii.tobs.desc()\
                                                ).all()

In [50]:
#Query to return all of the temp from 'Waihee' during 2016-08-23 and 2017-08-23.

query_temp = session.query(Hawaii.name, Hawaii.tobs, Hawaii.date_format).\
                filter(Hawaii.name == 'WAIHEE 837.5, HI US').\
                filter(Hawaii.date_format.between('2016-08-23 00:00:00.000000', '2017-08-23 00:00:00.000000')).all()

In [ ]:
def get_temp(early_date_str, late_date_str):
    early_date_dt = datetime.strptime(early_date_str, '%Y-%m-%d')
    late_date_dt = datetime.strptime(late_date_str, '%Y-%m-%d')
    early_date = early_date_dt + timedelta(days=-365)
    late_date = late_date_dt + timedelta(days=-365)
    query = 'SELECT date_format, tobs FROM hawaii_data WHERE (date_format BETWEEN ' + '\"' + str(early_date) + '\"' ' AND ' + '\"' + str(late_date) + '\"' + ')'
    df_temp = pd.read_sql(query, conn)
    ave_temp = df_temp['tobs'].mean()
    max_temp = df_temp['tobs'].max()
    min_temp = df_temp['tobs'].min()
    delta_temp = max_temp - min_temp
    return ave_temp, max_temp, min_temp, delta_temp

In [73]:
#Query to take two dates and return all of the temp from the previous year from all statios

def get_temp(early_date_str, late_date_str):
    from datetime import datetime, timedelta
    early_date_dt = datetime.strptime(early_date_str, '%Y-%m-%d')
    late_date_dt = datetime.strptime(late_date_str, '%Y-%m-%d')
    early_date = early_date_dt + timedelta(days=-365)
    late_date = late_date_dt + timedelta(days=-365)
    ave_temp = session.query(func.avg(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    max_temp = session.query(func.max(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    min_temp = session.query(func.min(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    delta_temp = session.query(func.max(Hawaii.tobs) - func.min(Hawaii.tobs)).filter(Hawaii.date_format.between(early_date, late_date)).all()
    return ave_temp, max_temp, min_temp, delta_temp

In [85]:
get_temp('2017-08-01', '2017-08-12')

([(78.21518987341773)], [(83)], [(71)], [(12)])

In [ ]:
app = Flask(__name__)
@app.route('/api/v1.0/<start>/<end>')
def precip():
    